This section will look to solve the Traveling Salesman Problem through the Miller-Tucker-Zemlin formulation. 

In [5]:
# Import packages
from gurobipy import *
import pandas as pd
import numpy as np
import math
from itertools import combinations,product

In [6]:
# Function will convert our datfile of x,y coordinates into dataframe: vertices
def TSP_vertices(file_name):
    data  = open(file_name, 'r')
    lines = data.readlines()[1:]
    x = []
    y = []
    for line in lines:
        p = line.split()
        x.append(float(p[0]))
        y.append(float(p[1]))
    data.close()
    # Transform into dataframe
    vertices = pd.DataFrame({'x':x, 'y':y})
    
    return vertices

In [7]:
# Will convert Euclidean distance for our coordinates as cost components for model
def euclideanDistMatrix(TSP):
    euclidean_cost = []
    for i in range(len(TSP)):
        row_distances = []
        for j in range(len(TSP)):
            row_distances.append(math.sqrt( (TSP['x'][i]-TSP['x'][j])**2 + (TSP['y'][i]-TSP['y'][j])**2 ))
        euclidean_cost.append(row_distances)
    return euclidean_cost 

In [8]:
def Miller_Tucker_Zemlin(file_name):

    # Turn .dat file into a cost matrix with all costs associated from traversing from i to j
    verticesDF = TSP_vertices(file_name)
    cost = euclideanDistMatrix(verticesDF)

    # Create a list V containing all vertices. Note: numbered 0 through n-1
    V = []
    for i in range(len(verticesDF)):
        V.append(i)
    
    # Create a list A of all tuples of arc (i,j) and U as indexes for variable u
    A = [(i, j) for i in V for j in V if i != j]
    U = [i for i in range(1, len(V))]

    # Instantiate the Model
    model = Model('MTZ')
    model.ModelSense = GRB.MINIMIZE

    # Instantiate the decision variable
    x = model.addVars(A, vtype = GRB.BINARY, name='e')
    u = model.addVars(U, lb = 0, vtype = GRB.CONTINUOUS, name='u')

    # Objective Function
    model.modelSense = GRB.MINIMIZE
    model.setObjective(quicksum(x[i,j]*cost[i][j] for i, j in A))

    # Constraint 1: one enters i
    model.addConstrs(quicksum(x[i,j] for j in V if i != j) == 1 for i in V)
    # Constraint 2: one exits i
    model.addConstrs(quicksum(x[j,i] for j in V if i != j) == 1 for i in V)
    # Constraint 3: 
    model.addConstrs(u[i] - u[j] + len(V)*x[i,j] <= len(V)-1 for i in V[1:] for j in V[1:] if i != j)
    # Solve the model. 
    model.Params.TimeLimit = 600
    model.optimize()
    print('The runtime was', model.Runtime, 'seconds.')

    ### UNCOMMENT LINE BELOW if you want to save a file of the formulation. Open with Microsoft Word.
    #model.write('MTZ.lp')

    # Print out postive decision varialbes
    #for v in model.getVars():
     #   if v.x > 0:
      #      print(v)
            
    return model

In [9]:
def Miller_Tucker_Zemlin_LR(file_name):

    # Turn .dat file into a cost matrix with all costs associated from traversing from i to j
    verticesDF = TSP_vertices(file_name)
    cost = euclideanDistMatrix(verticesDF)

    # Create a list V containing all vertices. Note: numbered 0 through n-1
    V = []
    for i in range(len(verticesDF)):
        V.append(i)
    
    # Create a list A of all tuples of arc (i,j) and U as indexes for variable u
    A = [(i, j) for i in V for j in V if i != j]
    U = [i for i in range(1, len(V))]

    # Instantiate the Model
    model = Model('MTZ')

    # Instantiate the decision variables. Do a linear relaxation on our variable x representing the arcs between 0 and 1. 
    x = model.addVars(A, lb = 0, ub = 1, vtype = GRB.CONTINUOUS, name='e')
    u = model.addVars(U, lb = 0, vtype = GRB.CONTINUOUS, name='u')

    # Objective Function
    model.modelSense = GRB.MINIMIZE
    model.setObjective(quicksum(x[i,j]*cost[i][j] for i, j in A))

    # Constraint 1: one enters i
    model.addConstrs(quicksum(x[i,j] for j in V if i != j) == 1 for i in V)
    # Constraint 2: one exits i
    model.addConstrs(quicksum(x[j,i] for j in V if i != j) == 1 for i in V)
    # Constraint 3: 
    model.addConstrs(u[i] - u[j] + len(V)*x[i,j] <= len(V)-1 for i in V[1:] for j in V[1:] if i != j)
    # Solve the model. 
    model.Params.TimeLimit = 600
    model.optimize()
    print('The runtime was', model.Runtime, 'seconds.')

    ### UNCOMMENT LINE BELOW if you want to save a file of the formulation. Open with Microsoft Word.
    # model.write('MTZ.lp')

    # Print out postive decision varialbes
    #for v in model.getVars():
     #   if v.x > 0:
      #      print(v)
            
    return model

The following are the Miller-Tucker-Zemlin formulation integer and Linear Relaxation solutions for two instances of size 15.

In [10]:
Miller_Tucker_Zemlin('TSP_instance_n_15_s_1.dat')

Using license file /Users/student/gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 212 rows, 224 columns and 966 nonzeros
Model fingerprint: 0xd50ba5d6
Variable types: 14 continuous, 210 integer (210 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve time: 0.01s
Presolved: 212 rows, 224 columns, 966 nonzeros
Variable types: 14 continuous, 210 integer (210 binary)

Root relaxation: objective 5.567906e+01, 39 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   55.67906    0   23          -   55.67906      -     -    0s
H    0     0                    

<gurobi.Model MIP instance MTZ: 212 constrs, 224 vars, Parameter changes: TimeLimit=600.0>

In [11]:
Miller_Tucker_Zemlin_LR('TSP_instance_n_15_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 212 rows, 224 columns and 966 nonzeros
Model fingerprint: 0xfede2098
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 211 rows, 224 columns, 952 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1738894e+01   1.568750e+01   0.000000e+00      0s
      33    5.5679061e+01   0.000000e+00   0.000000e+00      0s

Solved in 33 iterations and 0.03 seconds
Optimal objective  5.567906100e+01
The runtime was 0.0357820987701416 seconds.


<gurobi.Model Continuous instance MTZ: 212 constrs, 224 vars, Parameter changes: TimeLimit=600.0>

In [12]:
Miller_Tucker_Zemlin('TSP_instance_n_15_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 212 rows, 224 columns and 966 nonzeros
Model fingerprint: 0xdbe108c9
Variable types: 14 continuous, 210 integer (210 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [8e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve time: 0.01s
Presolved: 212 rows, 224 columns, 966 nonzeros
Variable types: 14 continuous, 210 integer (210 binary)

Root relaxation: objective 5.145035e+01, 36 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   51.45035    0   22          -   51.45035      -     -    0s
H    0     0                     133.2333128   51.45035  61.4%     -    0s
     0     0   51.98715    0   22  133.23331   51

<gurobi.Model MIP instance MTZ: 212 constrs, 224 vars, Parameter changes: TimeLimit=600.0>

In [13]:
Miller_Tucker_Zemlin_LR('TSP_instance_n_15_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 212 rows, 224 columns and 966 nonzeros
Model fingerprint: 0xfc3e0b09
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [8e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 211 rows, 224 columns, 952 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.5739812e+01   9.750000e+00   0.000000e+00      0s
      31    5.1450354e+01   0.000000e+00   0.000000e+00      0s

Solved in 31 iterations and 0.03 seconds
Optimal objective  5.145035362e+01
The runtime was 0.03203606605529785 seconds.


<gurobi.Model Continuous instance MTZ: 212 constrs, 224 vars, Parameter changes: TimeLimit=600.0>

The following are the Miller-Tucker-Zemlin formulation integer and Linear Relaxation solutions for two instances of size 20.

In [14]:
Miller_Tucker_Zemlin('TSP_instance_n_20_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 382 rows, 399 columns and 1786 nonzeros
Model fingerprint: 0xd71c23ac
Variable types: 19 continuous, 380 integer (380 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve time: 0.01s
Presolved: 382 rows, 399 columns, 1786 nonzeros
Variable types: 19 continuous, 380 integer (380 binary)

Root relaxation: objective 4.725842e+01, 65 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   47.25842    0   24          -   47.25842      -     -    0s
     0     0   59.08305    0   40          -   59.08305      -     -    0s
     0     0   59.08305    0   40          -   

<gurobi.Model MIP instance MTZ: 382 constrs, 399 vars, Parameter changes: TimeLimit=600.0>

In [15]:
Miller_Tucker_Zemlin_LR('TSP_instance_n_20_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 382 rows, 399 columns and 1786 nonzeros
Model fingerprint: 0x96bf8cc1
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.02s
Presolved: 381 rows, 399 columns, 1767 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.8979196e+01   1.700000e+01   0.000000e+00      0s
      66    4.7258423e+01   0.000000e+00   0.000000e+00      0s

Solved in 66 iterations and 0.03 seconds
Optimal objective  4.725842260e+01
The runtime was 0.03602004051208496 seconds.


<gurobi.Model Continuous instance MTZ: 382 constrs, 399 vars, Parameter changes: TimeLimit=600.0>

In [16]:
Miller_Tucker_Zemlin('TSP_instance_n_20_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 382 rows, 399 columns and 1786 nonzeros
Model fingerprint: 0xe88fc0b6
Variable types: 19 continuous, 380 integer (380 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [6e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve time: 0.01s
Presolved: 382 rows, 399 columns, 1786 nonzeros
Variable types: 19 continuous, 380 integer (380 binary)

Root relaxation: objective 6.123296e+01, 59 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   61.23296    0   34          -   61.23296      -     -    0s
     0     0   65.99023    0   43          -   65.99023      -     -    0s
     0     0   68.77515    0   36          -   

<gurobi.Model MIP instance MTZ: 382 constrs, 399 vars, Parameter changes: TimeLimit=600.0>

In [17]:
Miller_Tucker_Zemlin_LR('TSP_instance_n_20_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 382 rows, 399 columns and 1786 nonzeros
Model fingerprint: 0x188e429c
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [6e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 381 rows, 399 columns, 1767 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6713347e+01   1.712500e+01   0.000000e+00      0s
      56    6.1232956e+01   0.000000e+00   0.000000e+00      0s

Solved in 56 iterations and 0.03 seconds
Optimal objective  6.123295569e+01
The runtime was 0.03321504592895508 seconds.


<gurobi.Model Continuous instance MTZ: 382 constrs, 399 vars, Parameter changes: TimeLimit=600.0>

The following are the Miller-Tucker-Zemlin formulation integer and Linear Relaxation solutions for two instances of size 25.

In [18]:
Miller_Tucker_Zemlin('TSP_instance_n_25_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 602 rows, 624 columns and 2856 nonzeros
Model fingerprint: 0xb5febece
Variable types: 24 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve time: 0.02s
Presolved: 602 rows, 624 columns, 2856 nonzeros
Variable types: 24 continuous, 600 integer (600 binary)

Root relaxation: objective 5.736183e+01, 77 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   57.36183    0   29          -   57.36183      -     -    0s
     0     0   71.20805    0   46          -   71.20805      -     -    0s
     0     0   71.20805    0   46          -   

<gurobi.Model MIP instance MTZ: 602 constrs, 624 vars, Parameter changes: TimeLimit=600.0>

In [19]:
Miller_Tucker_Zemlin_LR('TSP_instance_n_25_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 602 rows, 624 columns and 2856 nonzeros
Model fingerprint: 0x8ca2a05f
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.03s
Presolved: 601 rows, 624 columns, 2832 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.8113668e+01   1.562500e+01   0.000000e+00      0s
      66    5.7361830e+01   0.000000e+00   0.000000e+00      0s

Solved in 66 iterations and 0.04 seconds
Optimal objective  5.736183007e+01
The runtime was 0.04853487014770508 seconds.


<gurobi.Model Continuous instance MTZ: 602 constrs, 624 vars, Parameter changes: TimeLimit=600.0>

In [20]:
Miller_Tucker_Zemlin('TSP_instance_n_25_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 602 rows, 624 columns and 2856 nonzeros
Model fingerprint: 0x6e394675
Variable types: 24 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve time: 0.01s
Presolved: 602 rows, 624 columns, 2856 nonzeros
Variable types: 24 continuous, 600 integer (600 binary)

Root relaxation: objective 6.426216e+01, 74 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   64.26216    0   36          -   64.26216      -     -    0s
     0     0   75.29922    0   54          -   75.29922      -     -    0s
     0     0   75.30087    0   54          -   

<gurobi.Model MIP instance MTZ: 602 constrs, 624 vars, Parameter changes: TimeLimit=600.0>

In [21]:
Miller_Tucker_Zemlin_LR('TSP_instance_n_25_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 602 rows, 624 columns and 2856 nonzeros
Model fingerprint: 0xdd3bbf79
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.02s
Presolved: 601 rows, 624 columns, 2832 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6921931e+01   1.975000e+01   0.000000e+00      0s
      53    6.4262156e+01   0.000000e+00   0.000000e+00      0s

Solved in 53 iterations and 0.05 seconds
Optimal objective  6.426215614e+01
The runtime was 0.057734012603759766 seconds.


<gurobi.Model Continuous instance MTZ: 602 constrs, 624 vars, Parameter changes: TimeLimit=600.0>

The following are the Miller-Tucker-Zemlin formulation integer and Linear Relaxation solutions for two instances of size 30.

In [22]:
Miller_Tucker_Zemlin('TSP_instance_n_30_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 872 rows, 899 columns and 4176 nonzeros
Model fingerprint: 0x42be7b05
Variable types: 29 continuous, 870 integer (870 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve time: 0.02s
Presolved: 872 rows, 899 columns, 4176 nonzeros
Variable types: 29 continuous, 870 integer (870 binary)

Root relaxation: objective 6.857550e+01, 106 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   68.57550    0   47          -   68.57550      -     -    0s
     0     0   82.42392    0   61          -   82.42392      -     -    0s
     0     0   83.08417    0   52          -  

<gurobi.Model MIP instance MTZ: 872 constrs, 899 vars, Parameter changes: TimeLimit=600.0>

In [23]:
Miller_Tucker_Zemlin_LR('TSP_instance_n_30_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 872 rows, 899 columns and 4176 nonzeros
Model fingerprint: 0x5fc60919
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.03s
Presolved: 871 rows, 899 columns, 4147 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.4424974e+01   2.025000e+01   0.000000e+00      0s
      95    6.8575499e+01   0.000000e+00   0.000000e+00      0s

Solved in 95 iterations and 0.05 seconds
Optimal objective  6.857549944e+01
The runtime was 0.06148195266723633 seconds.


<gurobi.Model Continuous instance MTZ: 872 constrs, 899 vars, Parameter changes: TimeLimit=600.0>

In [24]:
Miller_Tucker_Zemlin('TSP_instance_n_30_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 872 rows, 899 columns and 4176 nonzeros
Model fingerprint: 0xdf16a147
Variable types: 29 continuous, 870 integer (870 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve time: 0.01s
Presolved: 872 rows, 899 columns, 4176 nonzeros
Variable types: 29 continuous, 870 integer (870 binary)

Root relaxation: objective 5.769402e+01, 110 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   57.69402    0   40          -   57.69402      -     -    0s
H    0     0                     287.3057702   57.69402  79.9%     -    0s
     0     0   81.21644    0   60  287.30577  

<gurobi.Model MIP instance MTZ: 872 constrs, 899 vars, Parameter changes: TimeLimit=600.0>

In [25]:
Miller_Tucker_Zemlin_LR('TSP_instance_n_30_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 872 rows, 899 columns and 4176 nonzeros
Model fingerprint: 0x04c4afc5
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.02s
Presolved: 871 rows, 899 columns, 4147 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6994596e+01   1.637500e+01   0.000000e+00      0s
     127    5.7694022e+01   0.000000e+00   0.000000e+00      0s

Solved in 127 iterations and 0.05 seconds
Optimal objective  5.769402174e+01
The runtime was 0.05445599555969238 seconds.


<gurobi.Model Continuous instance MTZ: 872 constrs, 899 vars, Parameter changes: TimeLimit=600.0>

The following are the Miller-Tucker-Zemlin formulation integer and Linear Relaxation solutions for two instances of size 50.

In [26]:
Miller_Tucker_Zemlin('TSP_instance_n_50_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 2452 rows, 2499 columns and 11956 nonzeros
Model fingerprint: 0x15066e93
Variable types: 49 continuous, 2450 integer (2450 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve time: 0.04s
Presolved: 2452 rows, 2499 columns, 11956 nonzeros
Variable types: 49 continuous, 2450 integer (2450 binary)

Root relaxation: objective 9.477304e+01, 176 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   94.77304    0   90          -   94.77304      -     -    0s
     0     0  106.71075    0   98          -  106.71075      -     -    0s
     0     0  106.74395    0   99   

<gurobi.Model MIP instance MTZ: 2452 constrs, 2499 vars, Parameter changes: TimeLimit=600.0>

In [27]:
Miller_Tucker_Zemlin_LR('TSP_instance_n_50_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 2452 rows, 2499 columns and 11956 nonzeros
Model fingerprint: 0x346fc415
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.02s
Presolved: 2451 rows, 2499 columns, 11907 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.5466798e+01   3.275000e+01   0.000000e+00      0s
     179    9.4773036e+01   0.000000e+00   0.000000e+00      0s

Solved in 179 iterations and 0.05 seconds
Optimal objective  9.477303611e+01
The runtime was 0.05704188346862793 seconds.


<gurobi.Model Continuous instance MTZ: 2452 constrs, 2499 vars, Parameter changes: TimeLimit=600.0>

In [28]:
Miller_Tucker_Zemlin('TSP_instance_n_50_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 2452 rows, 2499 columns and 11956 nonzeros
Model fingerprint: 0x0d49c118
Variable types: 49 continuous, 2450 integer (2450 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve time: 0.03s
Presolved: 2452 rows, 2499 columns, 11956 nonzeros
Variable types: 49 continuous, 2450 integer (2450 binary)

Root relaxation: objective 8.651538e+01, 159 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   86.51538    0   73          -   86.51538      -     -    0s
     0     0   92.82769    0  103          -   92.82769      -     -    0s
     0     0   93.38587    0   99   

 146499 59552  106.30894  135   26  108.46219  104.55951  3.60%  16.6  295s
 150713 62508  105.19793  124   39  108.46219  104.57880  3.58%  16.6  300s
 154606 65460  105.73647  148   39  108.46219  104.59137  3.57%  16.6  305s
 158504 68133  105.76682  164   45  108.46219  104.60589  3.56%  16.6  310s
 162872 71173  107.67964  147   22  108.46219  104.62018  3.54%  16.6  315s
H166557 69503                     107.9036250  104.63089  3.03%  16.6  318s
 167399 69948  107.78963  143   20  107.90362  104.63457  3.03%  16.6  321s
 170573 71987  107.12631  188   32  107.90362  104.64955  3.02%  16.6  325s
 173822 73822  105.60376  128   91  107.90362  104.66640  3.00%  16.6  330s
 177031 75519  106.18478  132   61  107.90362  104.68023  2.99%  16.6  335s
 180419 77606     cutoff  139       107.90362  104.69336  2.98%  16.7  340s
 184417 79849  107.05286  134   97  107.90362  104.70936  2.96%  16.7  345s
 188107 82145 infeasible  160       107.90362  104.72510  2.95%  16.7  350s
 192002 8441

<gurobi.Model MIP instance MTZ: 2452 constrs, 2499 vars, Parameter changes: TimeLimit=600.0>

In [29]:
Miller_Tucker_Zemlin_LR('TSP_instance_n_50_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 2452 rows, 2499 columns and 11956 nonzeros
Model fingerprint: 0x1f55d68e
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.03s
Presolved: 2451 rows, 2499 columns, 11907 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.4225866e+01   4.287500e+01   0.000000e+00      0s
     116    8.6515384e+01   0.000000e+00   0.000000e+00      0s

Solved in 116 iterations and 0.06 seconds
Optimal objective  8.651538385e+01
The runtime was 0.06833696365356445 seconds.


<gurobi.Model Continuous instance MTZ: 2452 constrs, 2499 vars, Parameter changes: TimeLimit=600.0>

The following are the Miller-Tucker-Zemlin formulation integer and Linear Relaxation solutions for two instances of size 70.

In [30]:
Miller_Tucker_Zemlin('TSP_instance_n_70_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 4832 rows, 4899 columns and 23736 nonzeros
Model fingerprint: 0x47b3c05f
Variable types: 69 continuous, 4830 integer (4830 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Presolve time: 0.05s
Presolved: 4832 rows, 4899 columns, 23736 nonzeros
Variable types: 69 continuous, 4830 integer (4830 binary)

Root relaxation: objective 1.108201e+02, 240 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  110.82006    0  110          -  110.82006      -     -    0s
     0     0  125.43627    0  139          -  125.43627      -     -    0s
     0     0  126.05365    0  137   

<gurobi.Model MIP instance MTZ: 4832 constrs, 4899 vars, Parameter changes: TimeLimit=600.0>

In [31]:
Miller_Tucker_Zemlin_LR('TSP_instance_n_70_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 4832 rows, 4899 columns and 23736 nonzeros
Model fingerprint: 0x54f04233
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.04s
Presolved: 4831 rows, 4899 columns, 23667 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.4768870e-01   1.350000e+02   0.000000e+00      0s
     265    1.1082006e+02   0.000000e+00   0.000000e+00      0s

Solved in 265 iterations and 0.09 seconds
Optimal objective  1.108200564e+02
The runtime was 0.09696292877197266 seconds.


<gurobi.Model Continuous instance MTZ: 4832 constrs, 4899 vars, Parameter changes: TimeLimit=600.0>

In [32]:
Miller_Tucker_Zemlin('TSP_instance_n_70_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 4832 rows, 4899 columns and 23736 nonzeros
Model fingerprint: 0xc81d9236
Variable types: 69 continuous, 4830 integer (4830 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Presolve time: 0.04s
Presolved: 4832 rows, 4899 columns, 23736 nonzeros
Variable types: 69 continuous, 4830 integer (4830 binary)

Root relaxation: objective 1.090040e+02, 256 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  109.00401    0  130          -  109.00401      -     -    0s
     0     0  126.97488    0  128          -  126.97488      -     -    1s
     0     0  127.07923    0  131   

 57268 15004     cutoff  128       132.03932  131.02368  0.77%  15.9  335s
 59657 15169  131.46443  152  134  132.03932  131.06175  0.74%  16.0  341s
 59669 15177  131.39496  134  188  132.03932  131.06175  0.74%  16.0  345s
 59681 15185  131.90657  127  175  132.03932  131.06175  0.74%  16.0  350s
 59690 15191  131.53473  145  175  132.03932  131.06175  0.74%  16.0  355s
 59697 15196  131.87603  120  192  132.03932  131.06175  0.74%  16.0  360s
 59705 15201  131.50418  146   91  132.03932  131.06175  0.74%  16.0  365s
 59715 15208  131.51365  136  177  132.03932  131.06175  0.74%  16.0  370s
 59721 15212  131.49463  139  178  132.03932  131.06175  0.74%  15.9  375s
 59727 15216  131.66679  153  179  132.03932  131.06175  0.74%  15.9  380s
 59733 15220  131.41200  169  175  132.03932  131.06175  0.74%  15.9  385s
 59738 15223  131.81115  133  179  132.03932  131.06175  0.74%  15.9  390s
 59744 15227  131.42038  121  178  132.03932  131.06175  0.74%  15.9  395s
 59748 15230  131.78288  

<gurobi.Model MIP instance MTZ: 4832 constrs, 4899 vars, Parameter changes: TimeLimit=600.0>

In [33]:
Miller_Tucker_Zemlin_LR('TSP_instance_n_70_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 4832 rows, 4899 columns and 23736 nonzeros
Model fingerprint: 0x403b6578
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.05s
Presolved: 4831 rows, 4899 columns, 23667 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.7459361e-01   1.350000e+02   0.000000e+00      0s
     306    1.0900401e+02   0.000000e+00   0.000000e+00      0s

Solved in 306 iterations and 0.09 seconds
Optimal objective  1.090040080e+02
The runtime was 0.09775996208190918 seconds.


<gurobi.Model Continuous instance MTZ: 4832 constrs, 4899 vars, Parameter changes: TimeLimit=600.0>